# 1.Import dependencies

In [3]:
import cv2
import numpy as np

# 2. Import Models

In [4]:
import joblib

FC_MODEL = joblib.load("../../models/Face Detector/FC_F640_A87.pkl")
FD_MODEL = joblib.load("../../models/Face Detector/Face_Detecting_Model_Trial_I.pkl")

# 3. Define Helper Functions

In [5]:
VIDEO_CAPTURE_FRAME_SIZE = (640, 480)

FC_FRAME_SIZE = 640
FD_FRAME_SIZE = 640

FC_HOG_WIN_SIZE = (FC_FRAME_SIZE, FC_FRAME_SIZE) 
FC_HOG_BLOCK_SIZE = (32, 32)
FC_HOG_BLOCK_STRIDE = (32, 32)
FC_HOG_CELL_SIZE = (8, 8)
FC_HOG_NUM_BINS = 9

FC_HOG = cv2.HOGDescriptor(FC_HOG_WIN_SIZE, FC_HOG_BLOCK_SIZE, FC_HOG_BLOCK_STRIDE, FC_HOG_CELL_SIZE, FC_HOG_NUM_BINS)
FD_HOG = FC_HOG

def extract_features_for_fc(frame) -> list[list]:
    """
    returns the feature vector for the face classifier
    """
    gray_frame = cv2.resize(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY), (FC_FRAME_SIZE, FC_FRAME_SIZE))
    hog = FC_HOG.compute(gray_frame).flatten() # gray_frame[64:577, 64:577] -> crops the centermost 512x512 window
    
    return [hog / hog.max()]

def extract_features_for_fd(frame) -> list[list]:
    """
    returns the feature vector for the face detector
    """
    gray_frame = cv2.resize(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY), (FD_FRAME_SIZE, FD_FRAME_SIZE))
    hog = FD_HOG.compute(gray_frame).flatten() # gray_frame[64:577, 64:577] -> crops the centermost 512x512 window
    
    return [hog / hog.max()]

# 4. Main Application

In [16]:
def detect_faces(image, classifier, threshold=0.5, window_size=(24, 24), stride=4):
    detected_faces = []

    height, width = image.shape[:2]

    # Slide the window over the image
    for y in range(0, height - window_size[1] + 1, stride):
        for x in range(0, width - window_size[0] + 1, stride):
            # Extract the region of interest
            roi = image[y:y + window_size[1], x:x + window_size[0]]

            # Extract Haar-like features for the region
            features = extract_haar_features(roi)  # You need to implement this function

            # Use the trained classifier to predict
            prediction = classifier.predict_proba([features])[0, 1]

            # If the prediction exceeds the threshold, consider it as a face
            if prediction > threshold:
                detected_faces.append((x, y, window_size[0], window_size[1]))

    return detected_faces


camera = cv2.VideoCapture(0)

while camera.isOpened():

    ret, frame = camera.read()

    fc_features = extract_features_for_fc(np.copy(frame))

    is_face: bool = FC_MODEL.predict(fc_features)

    if is_face:

        fd_features = extract_features_for_fd(np.copy(frame))
        [[x0_face, y0_face, x1_face, y1_face]] = FD_MODEL.predict(fd_features)

        x0_face = int(x0_face * frame.shape[0])
        y0_face = int(y0_face * frame.shape[1])
        x1_face = int(x1_face * frame.shape[0])
        y1_face = int(y1_face * frame.shape[1])

        # Draw a green rectangle
        cv2.rectangle(frame, (y0_face, x0_face), (y1_face, x1_face), (0, 255, 0), 2)
    else:
        # Draw a red rectangle
        cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), (0, 0, 255), 5)

    cv2.imshow("Stream", frame)

    if cv2.waitKey(25) & 0xFF == ord('q'):
        print(frame.shape)
        break

camera.release()
cv2.destroyAllWindows()

(480, 640, 3)
